# Ttest for  sources

Script of Nikita Tyulenev **source_plot_get_full_stc.py** was used

In [7]:
import os
import mne
import numpy as np
import pandas as pd
from scipy import stats
from statsmodels.stats import multitest as mul

In [36]:
# This code sets an environment variable called SUBJECTS_DIR
os.environ['SUBJECTS_DIR'] = '/net/server/data/Archive/prob_learn/freesurfer'
subjects_dir = '/net/server/data/Archive/prob_learn/freesurfer'

Функция для отображения значимых отличий в пространсве источников (на мозге), т.е. цветом будет рисоваться значимость: если t критерий меньше нуля, то цвет синий, чем насыщенее цвет, тем значимость больше, тоже самое в обратную сторону (если t критерий больше 0)

In [3]:
def signed_p_val(t, p_val):
    if t >= 0:
        return 1 - p_val
    else:
        return -(1 - p_val)
    
vect_signed_p_val = np.vectorize(signed_p_val)     

##### сравниваем фидбеки внутри риска

In [43]:
session = ["risk_fb_cur_positive", "risk_fb_cur_negative"]

subjects = pd.read_csv('/home/vtretyakova/Рабочий стол/probability_learning/sources/subj_list.csv')['subj_list'].tolist()

subjects.remove('P062') #without MRI
subjects.remove('P052') # bad segmentation, попробовали запустить freesurfer еще раз не помогло

subjects.remove('P032') #ValueError: dimension mismatch - попробовали запустить freesurfer еще раз не помогло. Надо разбираться

subjects.remove('P045') #RuntimeError: Could not find neighbor for vertex 4395 / 10242 когда использую для задания пространства источников src, ico5 вместо oct6


In [44]:
inter = [-0.800, 2.400] #Time interval

In [11]:
data_dir = '/net/server/data/Archive/prob_learn/vtretyakova/sources/beta_16_30/beta_16_30_stc_fsaverage_ave_into_subj'

### download donor

In [45]:
stc_test = mne.read_source_estimate('/net/server/data/Archive/prob_learn/vtretyakova/sources/beta_16_30/beta_16_30_stc_fsaverage_ave_into_subj/P001_risk_fb_cur_negative', 'fsaverage').crop(tmin=inter[0], tmax=inter[1], include_tmax=True)

##### make empty dataframe and forler for output files

In [46]:
comp1_per_sub = np.zeros(shape=(len(subjects), stc_test.data.shape[0], stc_test.data.shape[1]))
comp2_per_sub = np.zeros(shape=(len(subjects), stc_test.data.shape[0], stc_test.data.shape[1]))
os.makedirs('/net/server/data/Archive/prob_learn/vtretyakova/sources/beta_16_30/test_risk_neg_vs_pos' , exist_ok = True)

# TTEST

In [47]:
for ind, subj in enumerate(subjects):
    print(ind + 1, subj)
    temp1 = mne.read_source_estimate(os.path.join(data_dir, "{0}_{1}-lh.stc".format(subj, session[0]))).crop(tmin=inter[0], tmax=inter[1], include_tmax=True)
    #temp1.resample(40)
    comp1_per_sub[ind, :, :] = temp1.data
    temp2 = mne.read_source_estimate(os.path.join(data_dir, "{0}_{1}-lh.stc".format(subj, session[1]))).crop(tmin=inter[0], tmax=inter[1], include_tmax=True)
    #temp2.resample(40)
    comp2_per_sub[ind, :, :] = temp2.data

1 P001
2 P002
3 P004
4 P006
5 P007
6 P008
7 P011
8 P014
9 P015
10 P016
11 P017
12 P019
13 P021
14 P022
15 P023
16 P024
17 P025
18 P028
19 P029
20 P030
21 P031
22 P033
23 P034
24 P035
25 P039
26 P040
27 P042
28 P043
29 P044
30 P047
31 P048
32 P053
33 P055
34 P057
35 P059
36 P060


In [21]:
print(comp1_per_sub.shape)
print(comp2_per_sub.shape)

(36, 20484, 21)
(36, 20484, 21)


In [48]:
t_stat, p_val = stats.ttest_rel(comp2_per_sub, comp1_per_sub, axis=0)
    #t_stat, p_val = stats.ttest_1samp(comp1_per_sub, popmean = 0, axis=0)
print(p_val.min(), p_val.mean(), p_val.max())
print(t_stat.min(), t_stat.mean(), t_stat.max())

8.114890798084246e-08 0.3757029067354736 0.9999992890056311
-3.772322099189191 0.9550223645604966 6.747333316896733


# full FDR 

In [49]:
width, height = p_val.shape
p_val_resh = p_val.reshape(width * height)
_, p_val = mul.fdrcorrection(p_val_resh)
p_val = p_val.reshape((width, height))
#t_stat = t_stat.reshape((width, height))

In [29]:
t_stat.shape

(20484, 21)

In [25]:
p_val.shape

(20484, 21)

In [50]:
p_val = vect_signed_p_val(t_stat, p_val)

In [51]:
print(p_val.min(), p_val.mean(), p_val.max())

-0.9396540982308526 0.3196111038583416 0.9692883138179542


### save stc with p values

In [52]:
stc_test.data = p_val
stc_test.save('/net/server/data/Archive/prob_learn/vtretyakova/sources/beta_16_30/test_risk_neg_vs_pos/beta_16_30_risk_fb_cur_pos_vs_neg_pval_full_fdr')

Writing STC to disk...
[done]


In [53]:
stc_test.plot(hemi='both')

Using control points [0.8830997  0.90892025 0.96794045]


### если нельзя сделать stc для p - value через присвоение, можно создать как объект

p_val_stc = mne.SourceEstimate(data = p_val, vertices = stc_test.vertices,  tmin = stc_test.tmin, tstep = stc_test.tstep)

p_val_stc.subject = 'fsaverage'   
p_val_stc.save(path)

# Если необхоидмо отрисовать цветом разницу сигналов

In [54]:
mean_data = (comp2_per_sub - comp1_per_sub).mean(axis=0) # усредняем между испытуемыми

In [55]:
stc_test.data = mean_data
stc_test.save('/net/server/data/Archive/prob_learn/vtretyakova/sources/beta_16_30/test_risk_neg_vs_pos/beta_16_30_risk_fb_cur_pos_vs_neg_mean_beta')

Writing STC to disk...
[done]


In [57]:
stc_test.plot(hemi='split')

Using control points [1.18596686 1.28215195 1.84813562]
